# data analysis and plotting results

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as pjoin
import seaborn as sns
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':'tight'}

## general definitions

In [ ]:
result_dir = "../raw-data/"
figures_dir = "../figures/results/"

In [ ]:
sns.set()
sns.set_style('ticks')
sns.set_context('paper',rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8})

In [ ]:
# Colour scheme

col_clean = tuple(x/255.0 for x in (50, 65, 75))
col_stylized = tuple(x/255.0 for x in (180, 160, 105))
col_combined = tuple(x/255.0 for x in (165, 30, 55))

In [ ]:
# plotting parameters

combined_plot_height = 7.0
combined_plot_width = 6.0

markersize = 16
fontsize = 24
linewidth = 2.5
labelsize = 18
legend_fontsize=18
axis_start = -0.2
axis_end = 5.3


In [ ]:
def get_results_coco_cityscapes(eval_output, task='bbox', metric='AP', verbose=False):

    results = np.zeros((20,6), dtype='float32')

    for i, distortion in enumerate(eval_output):
        for severity in eval_output[distortion]:
            mAP = eval_output[distortion][severity][task][metric]
            results[i, severity] = mAP
            if verbose:
                print(distortion, severity, mAP)
                
    return results

def get_results_voc(eval_output, verbose=False):

    results = np.zeros((20,6), dtype='float32')

    for i, distortion in enumerate(eval_output):
        for severity in eval_output[distortion]:
            mAP = np.mean([eval_output[distortion][severity][j]['ap'] for j in range(len(eval_output[distortion][severity]))])
            results[i, severity] = mAP
            if verbose:
                print(distortion, severity, mAP)
                
    return results

def get_distortions(eval_output):
    distortions = []
    for i, distortion in enumerate(eval_output):
        distortions.append(distortion.replace("_", " "))
    return distortions

In [ ]:
def plot_individual_results(data_A, data_B, data_C, plot_name, metric_name,
                            ylim, legend_loc=1, markersize=12, linewidth=2.0,
                            fontsize=15, legend_fontsize=11.3, labelsize=12.0,
                            data_name_A="standard data", data_name_B="stylized data",
                            data_name_C="combined data", color_A=col_clean,
                            color_B = col_stylized, color_C = col_combined):

    fig = plt.figure(figsize = (15.0, 20.0))
    plt.subplots_adjust(wspace = 0.3, hspace = 0.5)

    for i, distortion in enumerate(distortions[0:15]): # only plot first 15 distortions, not the validation distortions
        ax = plt.subplot(5,3,i+1)

        plt.plot(list(range(6)), data_A[i,:] * 100, 's-', zorder=2, color=color_A,
                 label=data_name_A, markersize=markersize, linewidth=linewidth)
        plt.plot(list(range(6)), data_B[i,:] * 100, '^-', zorder=1, color=color_B,
                 label=data_name_B, markersize=markersize, linewidth=linewidth)
        plt.plot(list(range(6)), data_C[i,:] * 100, 'o-', zorder=3, color=color_C,
                 label=data_name_C, markersize=markersize, linewidth=linewidth)


        plt.title(distortion, fontsize=fontsize)    
        if i >= 12: # plot xlabel only in last row
            plt.xlabel('corruption severity', fontsize=fontsize)
        if i % 3 == 0:
            plt.ylabel(metric_name, fontsize=fontsize)
        plt.axis([axis_start, axis_end, ylim[0], ylim[1]])
        if i == 0:
            legend = plt.legend(loc=legend_loc, fontsize=legend_fontsize, frameon=True, edgecolor="black")
            legend.get_frame().set_linewidth(1.0)

        ax.tick_params(axis='both', which='major', labelsize=labelsize)


    sns.despine(trim=True, offset=5)

    plt.show()

    fig.savefig(pjoin(figures_dir, plot_name), bbox_inches='tight')
    

### Pascal VOC

In [ ]:
with open(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_voc0712_results.pkl"), "rb") as f:
    voc_out = pickle.load(f)
with open(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_svoc0712_results.pkl"), "rb") as f:
    svoc_out = pickle.load(f)
with open(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_vocsvoc0712_results.pkl"), "rb") as f:
    voc_svoc_out = pickle.load(f)

In [ ]:
distortions = get_distortions(voc_out)

In [ ]:
voc_results = get_results_voc(voc_out)
svoc_results = get_results_voc(svoc_out)
voc_svoc_results = get_results_voc(voc_svoc_out)

In [ ]:
# absolute clean performance
print(np.mean(voc_results[0,0]), 
      np.mean(svoc_results[0,0]), 
      np.mean(voc_svoc_results[0,0]))

# absolute performance under corruption
print(np.mean(voc_results[:15,1:]), 
      np.mean(svoc_results[:15,1:]), 
      np.mean(voc_svoc_results[:15,1:]))

# realtive performance
print(np.mean(voc_results[:15,1:])/voc_results[0,0], 
      np.mean(svoc_results[:15,1:])/svoc_results[0,0], 
      np.mean(voc_svoc_results[:15,1:])/voc_svoc_results[0,0])

In [ ]:
def set_fontsize_helper(ax, fontsize):
    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fontsize)

In [ ]:
# Pascal VOC combined=averaged plot

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)
plt.plot(list(range(6)), np.mean(voc_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='standard data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(svoc_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(voc_svoc_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='combined data', markersize=markersize,
        linewidth=linewidth)

plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP50 in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 85])
ax.tick_params(axis='both', which='major', labelsize=labelsize)
legend = plt.legend(loc=3, fontsize=legend_fontsize, frameon=True, edgecolor="black")
legend.get_frame().set_linewidth(1.0)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, 'pascal_corruption_overall.pdf'), bbox_inches='tight')

In [ ]:
plot_individual_results(data_A = voc_results,
                        data_B=svoc_results,
                        data_C=voc_svoc_results,
                        plot_name="pascal_corruption_individual.pdf",
                        metric_name="mAP50 in %", ylim=[0, 85], legend_loc=3)

### MS COCO

In [ ]:
with open(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_coco_results.pkl"), "rb") as f:
    coco_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_scoco_results.pkl"), "rb") as f:
    scoco_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_cocoscoco_results.pkl"), "rb") as f:
    coco_scoco_out = pickle.load(f)

In [ ]:
distortions = get_distortions(coco_out)

In [ ]:
coco_results = get_results_coco_cityscapes(coco_out)
scoco_results = get_results_coco_cityscapes(scoco_out)
coco_scoco_results = get_results_coco_cityscapes(coco_scoco_out)

In [ ]:
# absolute clean performance
print(np.mean(coco_results[0,0]), 
      np.mean(scoco_results[0,0]), 
      np.mean(coco_scoco_results[0,0]))

# absolute performance under corruption
print(np.mean(coco_results[:15,1:]), 
      np.mean(scoco_results[:15,1:]), 
      np.mean(coco_scoco_results[:15,1:]))

# realtive performance
print(np.mean(coco_results[:15,1:])/coco_results[0,0], 
      np.mean(scoco_results[:15,1:])/scoco_results[0,0], 
      np.mean(coco_scoco_results[:15,1:])/coco_scoco_results[0,0])

In [ ]:
# MS COCO combined results

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)

plt.plot(list(range(6)), np.mean(coco_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='coco', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(scoco_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized coco', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(coco_scoco_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='combined', markersize=markersize,
        linewidth=linewidth)


plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 40])
ax.tick_params(axis='both', which='major', labelsize=labelsize)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, "coco_corruption_overall.pdf"), bbox_inches='tight')

In [ ]:
plot_individual_results(data_A = coco_results,
                        data_B=scoco_results,
                        data_C=coco_scoco_results,
                        plot_name="coco_corruption_individual.pdf",
                        metric_name="mAP in %", ylim=[0, 40])

### MS COCO Benchmark

In [ ]:
with open(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_r101_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_r101_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_x101_32x4d_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_x101_32x4d_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_x101_64x4d_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_x101_64x4d_out = pickle.load(f)
    
with open(pjoin(result_dir, "coco/mask_rcnn_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_mrcnn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/cascade_rcnn_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_crcnn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/cascade_mask_rcnn_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_cmrcnn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/retinanet_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_retina_r50_out = pickle.load(f)
    
with open(pjoin(result_dir, "coco/faster_rcnn_dconv_c3-c5_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_dcn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/faster_rcnn_dconv_c3-c5_x101_32x4d_fpn_1x_results.pkl"), "rb") as f:
    coco_frcnn_dcn_x101_32x4d_out = pickle.load(f)
with open(pjoin(result_dir, "coco/mask_rcnn_dconv_c3-c5_r50_fpn_1x_results.pkl"), "rb") as f:
    coco_mrcnn_dcn_r50_out = pickle.load(f)
with open(pjoin(result_dir, "coco/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_results.pkl"), "rb") as f:
    coco_htc_dcn_x101_64x4d_out = pickle.load(f)

In [ ]:
distortions = get_distortions(coco_frcnn_r50_out)

In [ ]:
coco_frcnn_r50_results = get_results_coco_cityscapes(coco_frcnn_r50_out)
coco_frcnn_r101_results = get_results_coco_cityscapes(coco_frcnn_r101_out)
coco_frcnn_x101_32x4d_results = get_results_coco_cityscapes(coco_frcnn_x101_32x4d_out)
coco_frcnn_x101_64x4d_results = get_results_coco_cityscapes(coco_frcnn_x101_64x4d_out)

coco_mrcnn_r50_results = get_results_coco_cityscapes(coco_mrcnn_r50_out)
coco_crcnn_r50_results = get_results_coco_cityscapes(coco_crcnn_r50_out)
coco_cmrcnn_r50_results = get_results_coco_cityscapes(coco_cmrcnn_r50_out)
coco_retina_r50_results = get_results_coco_cityscapes(coco_retina_r50_out)

coco_frcnn_dcn_r50_results = get_results_coco_cityscapes(coco_frcnn_dcn_r50_out)
coco_frcnn_dcn_x101_32x4d_results = get_results_coco_cityscapes(coco_frcnn_dcn_x101_32x4d_out)
coco_mrcnn_dcn_r50_results = get_results_coco_cityscapes(coco_mrcnn_dcn_r50_out)
coco_htc_dcn_x101_64x4d_results = get_results_coco_cityscapes(coco_htc_dcn_x101_64x4d_out)

coco_mrcnn_r50_results_segm = get_results_coco_cityscapes(coco_mrcnn_r50_out, task='segm')
coco_cmrcnn_r50_results_segm = get_results_coco_cityscapes(coco_cmrcnn_r50_out, task='segm')
coco_mrcnn_dcn_r50_results_segm = get_results_coco_cityscapes(coco_mrcnn_dcn_r50_out, task='segm')
coco_htc_dcn_x101_64x4d_results_segm = get_results_coco_cityscapes(coco_htc_dcn_x101_64x4d_out, task='segm')

In [ ]:
result_list = [coco_frcnn_r50_results, 
               coco_frcnn_r101_results, 
                coco_frcnn_x101_32x4d_results, 
                coco_frcnn_x101_64x4d_results,
                coco_frcnn_dcn_r50_results,
                coco_frcnn_dcn_x101_32x4d_results,
                coco_mrcnn_dcn_r50_results,
                coco_htc_dcn_x101_64x4d_results,
                coco_mrcnn_r50_results, 
                coco_crcnn_r50_results, 
                coco_cmrcnn_r50_results, 
                coco_retina_r50_results, 
                coco_mrcnn_r50_results_segm, 
                coco_cmrcnn_r50_results_segm, 
                coco_mrcnn_dcn_r50_results_segm,
                coco_htc_dcn_x101_64x4d_results_segm]

model_names = ['frcnn_r50', 
                'frcnn_r101', 
                'frcnn_x101_32x4d', 
                'frcnn_x101_64x4d',
                'frcnn_dcn_r50',
                'frcnn_dcn_x101_32x4d',
                'mrcnn_dcn_r50',
                'htc_dcn_x101_64x4d',
                'mrcnn_r50', 
                'crcnn_r50', 
                'cmrcnn_r50', 
                'retinanet_r50', 
                'mrcnn_r50_segm', 
                'cmrcnn_r50_segm',
                'mrcnn_dcn_r50_segm',
                'htc_dcn_x101_64x4d_segm']

colors = ['black','dark red', 'red', 'orange', 'dandelion']

In [ ]:
for i, results in enumerate(result_list):
    print('{}: clean:{:.1f} corr.:{:.1f} rel:{:.1f}%'.format(model_names[i], 
                                                           results[0,0] * 100, 
                                                           np.mean(results[:15,1:]) * 100, 
                                                           np.mean(results[:15,1:])/results[0,0] * 100))
    
    print('')

In [ ]:
# plot results for different backbones

# use different colour scheme to avoid confusion
col_resnet50 = col_clean
col_resnet101 = tuple(x/255.0 for x in (175, 110, 150))
col_resnext101 = tuple(x/255.0 for x in (210, 150, 0))

plot_individual_results(data_A = coco_frcnn_r50_results,
                        data_B=coco_frcnn_r101_results,
                        data_C=coco_frcnn_x101_64x4d_results,
                        color_A=col_resnet50,
                        color_B=col_resnet101,
                        color_C=col_resnext101,
                        data_name_A="ResNet-50",
                        data_name_B="ResNet-101",
                        data_name_C="ResNeXt-101",
                        plot_name="coco_corruption_backbones_individual.pdf",
                        metric_name="mAP in %", ylim=[-1, 45])

## Citscapes

In [ ]:
with open(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_city_results.pkl"), "rb") as f:
    city_out = pickle.load(f)
with open(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_scity_results.pkl"), "rb") as f:
    scity_out = pickle.load(f)
with open(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_cityscity_results.pkl"), "rb") as f:
    city_scity_out = pickle.load(f)

distortions = get_distortions(city_out)

In [ ]:
city_results = get_results_coco_cityscapes(city_out)
scity_results = get_results_coco_cityscapes(scity_out)
city_scity_results = get_results_coco_cityscapes(city_scity_out)

In [ ]:
# absolute clean performance
print(np.mean(city_results[0,0]), 
      np.mean(scity_results[0,0]), 
      np.mean(city_scity_results[0,0]))

# absolute performance under corruption
print(np.mean(city_results[:15,1:]), 
      np.mean(scity_results[:15,1:]), 
      np.mean(city_scity_results[:15,1:]))

    # realtive performance
print(np.mean(city_results[:15,1:])/city_results[0,0], 
      np.mean(scity_results[:15,1:])/scity_results[0,0], 
      np.mean(city_scity_results[:15,1:])/city_scity_results[0,0])

In [ ]:
# Cityscapes combined results

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)

plt.plot(list(range(6)), np.mean(city_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='standard data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(scity_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(city_scity_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='including stylized data', markersize=markersize,
        linewidth=linewidth)


plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 40])
ax.tick_params(axis='both', which='major', labelsize=labelsize)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, "cityscapes_corruption_overall.pdf"), bbox_inches='tight')

In [ ]:
plot_individual_results(clean_data = city_results,
                        stylized_data=scity_results,
                        combined_data=city_scity_results,
                        plot_name="cityscapes_corruption_individual.pdf",
                        metric_name="mAP in %", ylim=[-1, 40])